In [5]:
from ipywidgets import interact, widgets

In [11]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import re
from IPython.display import display
import plotly.express as px
import plotly.graph_objects as go
from PIL import Image
import plotly.express as px
import matplotlib.image as mpimg

#from dash import Dash, dcc, html
PROJECT_PATH = ".../../Milestone1/"
DATA_PATH = f"../data/IFT6758_Data"
sys.path.append(PROJECT_PATH)

from features.tidy_data import tidy_data

def load_tidy_data(year, season):
    path = DATA_PATH
    
    # Define game_type based on season
    game_type = '02' if season.lower() == 'regular' else '03'

    all_data = []

    # List all files in the directory
    year = str(year)
    files_in_directory = os.listdir(os.path.join(path, year, season))

    for file_name in files_in_directory:
        if file_name.endswith(".json"):
            # Extract the game_id from the file name (assuming the file names follow the pattern)
            game_id = int(file_name.split(".")[0])

            # Load the data and append it to the list
            game_data = tidy_data(path, year, season, game_id)
            if game_data is not None:
                all_data.append(game_data)

    # Concatenate all the data in the list
    if all_data:
        all_data = pd.concat(all_data)
    else:
        # Handle the case where no data was loaded
        print(f"No data found for year {year} and game type {game_type}")
        return None

    return all_data

# Function to load the rink image
def load_rink_image(image_path):
    return Image.open(image_path)

# Define offensive zone coordinates (adjust these based on your specific rink dimensions)
offensive_zone_x_min = 0
offensive_zone_x_max = 100
offensive_zone_y_min = -42.5
offensive_zone_y_max = 42.5



# Function to create a shot map on the rink image for a selected team
def create_shot_map_for_team(data, rink_image, selected_team):
    fig = go.Figure()


    # Define scale factors based on the rink image dimensions and scaling
    scale_factor_x = 1 #38.81 / 6100  # Assuming rink width is 200 units
    scale_factor_y = 1 #16.47 / 3000  # Assuming rink height is 85 units

    team_data = data[data['team'] == selected_team]
    X = []
    Y = []
    X_orig = []
    Y_orig = []
    for _, shot in team_data.iterrows():
        x = shot['x_coordinate']
        y = shot['y_coordinate']

        X_orig.append(x)
        Y_orig.append(y)
        if shot['goal_location'] == "Left":
            x = -x
            y = -y
        color = 'red' if shot['eventType'] == 'Goal' else 'blue'
        #print (x,y)
        # Check if the shot is in the offensive zone
        if (offensive_zone_x_min <= x <= offensive_zone_x_max) and (offensive_zone_y_min <= y <= offensive_zone_y_max):
            # Calculate adjusted coordinates with offsets
            x_adjusted = x * scale_factor_x
            y_adjusted = y * scale_factor_y

            X.append(x_adjusted)
            Y.append(y_adjusted)

        # Add a scatter point to the figure
       #fig.add_trace(go.Scatter(x=[x_adjusted], y=[y_adjusted], mode='markers', marker=dict(color=color, size=10)))
    if X_orig and X_orig != None:
        print("max in X {}".format(max(X_orig)))
        print("max in Y {}".format(max(Y_orig)))

        print("min in X {}".format(min(X_orig)))
        print("min in Y {}".format(min(Y_orig)))

    sns.set_style("white")
    #sns.cubehelix_palette(as_cmap=True)
    sns.kdeplot(x=X, y=Y,cmap='icefire', shade=True, bw_adjust=.5,cbar=True,alpha=0.6)
    plt.ylim(-42.5,42.5)
    plt.xlim(-100,100)
    plt.imshow(rink_image, extent=[-100,100,-42.5, 42.5], aspect='auto')

    plt.show()
    #plt.scatter(x=X,y=Y)

    # Add surface trace
    #fig.add_trace(go.Surface(z=df.values.tolist(), colorscale="Viridis"))
    # Customize the figure layout
    fig.update_layout(
        width=800,
        height=900,
        autosize=False,
        margin=dict(t=0, b=0, l=0, r=0),
        template="plotly_white",
        title=f"Shot Map for {selected_team}",
        xaxis_title="X Coordinate",
        yaxis_title="Y Coordinate",
    )

    # Display the rink image as a background image
    fig.add_layout_image(
        source=rink_image,
        xref="x",
        yref="y",
        x=0,
        y=85,  # Adjust this value based on the image positioning
        sizex=200,
        sizey=85,
        opacity=1,
        layer="below"
    )

    return fig

def run_viz(year, season, team):
    data = load_tidy_data(year, season)
    unique_teams = data['team'].unique()

    #rink_image_path = "nhl_rink.png"
    rink_image_path = "nhl_rink.png"
    rink_image = load_rink_image(rink_image_path)
    fig = create_shot_map_for_team(data, rink_image, team)

    fig.show()

team_name_dict = load_tidy_data(2020, 'regular')['team'].unique()

year_slider = widgets.IntSlider(min=2016, max=2020, step=1, description='Year:')
season_dropdown = widgets.Dropdown(options=['regular', 'playoffs'], description='Season:')
team_dropdown = widgets.Dropdown(options=team_name_dict, description='Team:')

interact(run_viz, year=year_slider, season=season_dropdown, team=team_dropdown)

interactive(children=(IntSlider(value=2016, description='Year:', max=2020, min=2016), Dropdown(description='Se…

<function __main__.run_viz(year, season, team)>